In [100]:
import numpy as np
import tulipy as ti
import pandas_datareader.data as web
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf
import math as m
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn import svm

In [52]:
style.use('ggplot')
# Date information
start_date = datetime.datetime(2015,1,1)
end_date = datetime.datetime(2022,1,3)
# Stock symbols
stock_symbols = pd.read_csv("tickers.csv", names = ["symbols"])
current_stock = stock_symbols.symbols[10]
print(current_stock)
# High Low Open Close data about the stock chosen
data = web.DataReader(current_stock, "yahoo", start_date, end_date)

ABB


In [53]:
def np_attributes(df):
    """
    Changed each column to numpy arrays
    [ high, low, close, volume, open]
    """
    close = df.Close.to_numpy()
    volume = df.Volume.to_numpy()
    high = df.High.to_numpy()
    low = df.Low.to_numpy()
    Open = df.Open.to_numpy()
    return [high,low, close, volume, Open]

In [101]:
def simple_moving_average(df, period):
    """
    DOES CHANGE SIZE MAKES NEW DF
    df - pandas dataframe
    period - how much the moving average is measured on
    return -  dataframe with simple moving average in it
    """
    np_vals = np_attributes(df)
    changed_df = df
    sma = ti.sma(np_vals[2], period)
    changed_df = changed_df.drop(changed_df.index[0:period-1])        
    changed_df["SMA"] = sma
    return changed_df

def accumulation_line(df, high, low, close, volume):
    """
    DOES NOT CHANGE SIZE ADDS TO GIVEN DF
    numpy arrays with the respective values and a
    pandas dataframe
    return - the dataframe with the accumulation line
    """
    ad = ti.ad(high, low, close, volume)
    df["AC Line"] = ad
    return df

def target_column(df, threshold, period):
    """DOES CHANGE SIZE OF DATAFRAME
        Makes target labels of 0 and 1
        depending on the threshold and period. 
        Both the threshold and period are ints
        
        Returns - the dataframe given (now smaller)
        with the target column in it
    """
    target = []
    profits = []
    i = 0
    while i<df.Close.shape[0]:
        if i+period >= df.Close.shape[0]:
            break
        profit =(df.Close[i]-df.Close[i+period])/df.Close[i]
        if profit<0:
            target.append(0)
            profits.append(profit)
        else:
            target.append(1)
            profits.append(profit)
        i+=1
    df = df.drop(df.index[0:period])
    df["Target"] = target
    return df

def fishers_transform(df, high, low, period = 0):
    """ CHANGES SIZE 
        INPUT - HIGH LOW, PERIOD
        OUTPUT - fisher_transfrom and its singal in the
        given dataframe
    
    """
    changed_df = df
    fisher,fisher_signal = ti.fisher(high, low, period)
    changed_df = changed_df.drop(changed_df.index[0:period-1])
    changed_df["fisher"] = fisher
    changed_df["fisher signal"] = fisher_signal
    return changed_df

def balance_of_power(df, high, low, Open, close):
    """Does not change size"""
    
    balance = ti.bop(Open, high, low, close)
    df["BalanceOP"] = balance
    return df
    

In [102]:
data = simple_moving_average(data,100)
attri =np_attributes(data)
data = accumulation_line(data, attri[0], attri[1],attri[2],attri[3])
data = fishers_transform(data,attri[0],attri[1], 5)
attri =np_attributes(data)
data = balance_of_power(data,attri[0], attri[1], attri[4], attri[2])
df = target_column(data, 0.05, 4)
print(df)

                 High        Low       Open      Close     Volume  Adj Close  \
Date                                                                           
2017-11-15  25.170000  25.040001  25.139999  25.090000  1217100.0  21.652308   
2017-11-16  25.360001  25.230000  25.270000  25.280001  1672500.0  21.816273   
2017-11-17  25.290001  25.020000  25.100000  25.230000  1617800.0  21.773125   
2017-11-20  25.459999  25.360001  25.370001  25.379999  1140200.0  21.902573   
2017-11-21  25.740000  25.469999  25.590000  25.549999  1408000.0  22.049278   
...               ...        ...        ...        ...        ...        ...   
2021-12-28  38.459999  38.270000  38.320000  38.380001   802400.0  38.380001   
2021-12-29  38.660000  38.320000  38.320000  38.650002   907000.0  38.650002   
2021-12-30  38.509998  38.090000  38.410000  38.139999  1370600.0  38.139999   
2021-12-31  38.349998  38.110001  38.110001  38.169998   706900.0  38.169998   
2022-01-03  38.660000  38.259998  38.619

In [103]:
x = df.loc[:, ["High","Low", "Close", "Volume", "SMA", "AC Line", "fisher", "fisher signal",
              "BalanceOP"]]
y = df.loc[:, ['Target']]
x_train = x[0:m.floor(x.shape[0] * 0.9)]
y_train = y[0:m.floor(y.shape[0]*0.9)]
x_test = x[m.ceil(x.shape[0] * 0.9):x.shape[0]]
y_test = y[m.ceil(y.shape[0] * 0.9):y.shape[0]]

In [104]:
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [117]:
model = tf.keras.models.Sequential()
model.add(Flatten(input_shape=(1, 9)))
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

In [118]:
adam = model.compile(optimizer='adagrad',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
model.fit(x_train, y_train, epochs=30, validation_data = (x_test,y_test))

Epoch 1/30
30/30 [==============================] - 1s 8ms/step - loss: 0.6912 - accuracy: 0.5459 - val_loss: 0.6915 - val_accuracy: 0.5385
Epoch 2/30
30/30 [==============================] - 0s 3ms/step - loss: 0.6897 - accuracy: 0.5427 - val_loss: 0.6909 - val_accuracy: 0.5385
Epoch 3/30
30/30 [==============================] - 0s 3ms/step - loss: 0.6882 - accuracy: 0.5641 - val_loss: 0.6905 - val_accuracy: 0.5385
Epoch 4/30
30/30 [==============================] - 0s 3ms/step - loss: 0.6894 - accuracy: 0.5684 - val_loss: 0.6903 - val_accuracy: 0.5385
Epoch 5/30
30/30 [==============================] - 0s 3ms/step - loss: 0.6879 - accuracy: 0.5598 - val_loss: 0.6902 - val_accuracy: 0.5385
Epoch 6/30
30/30 [==============================] - 0s 3ms/step - loss: 0.6870 - accuracy: 0.5566 - val_loss: 0.6902 - val_accuracy: 0.5385
Epoch 7/30
30/30 [==============================] - 0s 3ms/step - loss: 0.6851 - accuracy: 0.5588 - val_loss: 0.6902 - val_accuracy: 0.5385
Epoch 8/30
30/30 [==

In [119]:
# K nearest Neighbours
n=1
best_n = 1
best_acc = 0
while n<50:
    knn_model = KNeighborsClassifier(n_neighbors = n)
    knn_model.fit(x_train, y_train)
    predictions = knn_model.predict(x_test)
    i = 0
    total = 0
    correct = 0
    cor = []
    inc = []
    tot = []
    while i<len(predictions):
        if (predictions[i] == y_test[i] and predictions[i] == 1):
            total+=1
            correct+=1
            cor.append(y_test[i])
        elif predictions[i] == 1 and y_test[i]!= 1:
            total+=1
            inc.append(y_test[i])
        elif y_test[i] == 1:
            tot.append(y_test[i])
            total+=1
        i+=1
        
    if len(cor) == 0 or len(tot)==0:
        continue        
    else:
        acc = len(cor)/len(tot)
    if best_n == 1:
        best_n = n
        best_acc = acc
    elif best_acc < acc:
        best_acc = acc            
        best_n = n
    n+=1
print(best_acc, best_n)

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/pyth

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/pyth

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/pyth

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/pyth

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/pyth

...

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/pyth

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/pyth

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/pyth

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/pyth

KeyboardInterrupt: 

In [120]:
parameters = {"weights": ["uniform", "distance"]}
gridsearch = GridSearchCV(KNeighborsClassifier(n_neighbors = best_n), parameters)
gridsearch.fit(x_train, y_train)
predictions = gridsearch.predict(x_test)

/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pentaflouride/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/pentaflouride/anaconda3/lib/pyth

In [121]:
i = 0
total = 0
correct = 0
cor = []
inc = []
tot = []
    
while i<len(predictions):
    if (predictions[i] == y_test[i] and predictions[i] == 1):
            #print ((predictions[i], y_test[i]))
        total+=1
        correct+=1
        cor.append(y_test[i])
    elif predictions[i] == 1 and y_test[i]!= 1:
        total+=1
        inc.append(y_test[i])
    elif y_test[i] == 1:
        tot.append(y_test[i])
        total+=1
    i+=1
        
acc = len(cor)/len(tot)
acc

0.37142857142857144